In [3]:

import os
import numpy as np
import pandas as pd
import pma_python
from pma_python import core
import tqdm
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageDraw
import cv2
import matplotlib.pyplot as plt
import cv2
import os

from PIL import Image
Image.MAX_IMAGE_PIXELS = None 




def tile_wise_registration(reference_image, target_image,n, tile_size=(256, 256)):
    # Convert images to grayscale
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=n)

    # Find keypoints and descriptors with ORB
    keypoints1, descriptors1 = orb.detectAndCompute(reference_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(target_gray, None)

    # Create BFMatcher (Brute Force Matcher) object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)

    # Sort them in ascending order of distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched points
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate affine transformation
    transform_matrix = cv2.estimateAffinePartial2D(src_pts, dst_pts)[0]

    # Extract translation and rotation parameters
    tx = transform_matrix[0, 2]
    ty = transform_matrix[1, 2]
    rotation_rad = np.arctan2(transform_matrix[1, 0], transform_matrix[0, 0])

    # Convert rotation from radians to degrees
    rotation_deg = np.degrees(rotation_rad)

    # Apply the transformation to the target image
    registered_image = cv2.warpAffine(reference_image, transform_matrix, (target_image.shape[1], target_image.shape[0]))

    return registered_image, transform_matrix, tx, ty,rotation_deg








# Accept string inputs
ihc_o = input("Enter a pma_studio_ihc_file_path : ")
he_o = input("Enter a pma_studio_he_file_path: ")

# Accept numerical inputs and convert them to integers
X = int(input("Enter a coordinate for 'X': "))
Y = int(input("Enter a coordinate for 'Y': "))


region_size = int(input("Enter a region size"))

# Print the inputs
print("Inputs:")
print("'ihc' (string):", ihc_o)
print("'he' (string):", he_o)
print("'X' (number):", X)
print("'Y' (number):", Y)


sessionID = core.connect('http://10.100.76.62/core/', 'tmccomputpath', 'Z5XPLQ2I')

folder_name_prefix = he_o .split('/')[-2] 

folder_path = f"{folder_name_prefix}_region_{X}_{Y}"
os.makedirs(folder_path)
print(ihc_o)
ihc_8192   = core.get_region(ihc_o,x= X  ,y= Y ,width=region_size,height=region_size,scale= 1,rotation= 0 ,sessionID=sessionID)
output_path = os.path.join(folder_path, "ihc.tiff")
ihc_8192.save(output_path)
he_8192  = core.get_region(he_o,x= X  ,y= Y ,width=region_size,height=region_size,scale=1,rotation= 0,sessionID=sessionID)
output_path = os.path.join(folder_path, "he.tiff")
he_8192.save(output_path)

he = os.path.join(folder_path, 'he.tiff')
ihc = os.path.join(folder_path, 'ihc.tiff')

target_image = cv2.imread(ihc)
reference_image = cv2.imread(he)

registered_image, transformation_matrix,tx,ty,rotation_deg = tile_wise_registration(reference_image, target_image,50000)
registered_image_path = os.path.join(folder_path, "registered_X.tiff")
cv2.imwrite(registered_image_path, registered_image)
print(transformation_matrix,tx,ty,rotation_deg)

xnew = X+tx 
ynew = Y+ty
r = rotation_deg
print('xnew:',xnew,'ynew:',ynew,'rotation',r)

im=core.get_region(ihc_o,x= xnew ,y= ynew ,width=region_size,height=region_size,scale=1,rotation= r,sessionID=sessionID)
 
shifted_output_path = os.path.join(folder_path, "shifted_target.tiff")
im.save(shifted_output_path)






# Load the TIFF image
image = os.path.join(folder_path, "shifted_target.tiff")
image = cv2.imread(image, cv2.IMREAD_UNCHANGED)

# Get the dimensions of the image
height, width = image.shape[:2]

# Define the patch size
patch_size = 256

# Create the directory to save patches if it doesn't exist
save_dir = os.path.join(folder_path,'patches_ihc' )
os.makedirs(save_dir, exist_ok=True)

# Iterate over the image and extract patches
for y in range(0, height, patch_size):
    for x in range(0, width, patch_size):
        patch = image[y:y+patch_size, x:x+patch_size]
        patch_filename = os.path.join(save_dir, f'patch_{x}_{y}.png')
        cv2.imwrite(patch_filename, patch)




# Load the TIFF image
img = cv2.imread(he, cv2.IMREAD_UNCHANGED)

# Get the dimensions of the image
height, width = img.shape[:2]

# Define the patch size
patch_size = 256

# Create the directory to save patches if it doesn't exist
save_dir = os.path.join(folder_path,'patches_he' )
os.makedirs(save_dir, exist_ok=True)

# Iterate over the image and extract patches
for y in range(0, height, patch_size):
    for x in range(0, width, patch_size):
        patch = img[y:y+patch_size, x:x+patch_size]
        patch_filename = os.path.join(save_dir, f'patch_{x+X}_{y+Y}.png')
        cv2.imwrite(patch_filename, patch)



print('registered patches saved succesfully')


Enter a pma_studio_ihc_file_path :  CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302CD20/CAIB-T00000594OC01R01P0302CD20.svs
Enter a pma_studio_he_file_path:  CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302HE/CAIB-T00000594OC01R01P0302HE.svs
Enter a coordinate for 'X':  50000
Enter a coordinate for 'Y':  50000
Enter a region size 8192


Inputs:
'ihc' (string): CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302CD20/CAIB-T00000594OC01R01P0302CD20.svs
'he' (string): CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302HE/CAIB-T00000594OC01R01P0302HE.svs
'X' (number): 50000
'Y' (number): 50000
CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302CD20/CAIB-T00000594OC01R01P0302CD20.svs
[[ 9.99766263e-01  7.99847609e-03 -2.77911041e+03]
 [-7.99847609e-03  9.99766263e-01 -2.00996840e+03]] -2779.110413025152 -2009.9683955076343 -0.4583762851027285
xnew: 47220.889586974845 ynew: 47990.03160449237 rotation -0.4583762851027285
registered patches saved succesfully


In [ ]:
ihc = 'CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302CD20/CAIB-T00000594OC01R01P0302CD20.svs'
he = 'CAIB_WSI/CAIB/CAIB-T00000594OC/CAIB-T00000594OC01R01/CAIB-T00000594OC01R01P0302HE/CAIB-T00000594OC01R01P0302HE.svs'